In [ ]:
print(tf.__version__)
msg = tf.constant('TensorFlow 2.0 Hello World')
tf.print(msg)

2.5.0
TensorFlow 2.0 Hello World


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

%matplotlib inline

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras


In [ ]:
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy, mean_squared_error

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error as sklearn_mean_squared_error 
from sklearn.metrics import r2_score

In [ ]:
import tensorflow as tf

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals


**Загрузка данных**

In [ ]:
boston_dataset = load_boston()

In [ ]:
print(boston_dataset.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [ ]:
print(boston_dataset.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [ ]:
df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['target'] = pd.Series(boston_dataset.target)

**Анализ датасета**

In [ ]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [ ]:
df.shape

(506, 14)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


**Предобработка данных**

In [ ]:
X = df.drop(['target'], axis=1)
y = df['target']

In [ ]:
MinMaxSc = MinMaxScaler()
X_scaled = MinMaxSc.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.25, random_state=0)

**Построение модели**


In [ ]:
df = pd.DataFrame(columns=['no. layers', 'no. neurons', 'no. epochs', 'optimizer','MAE', 'MSE'])

**Кол-во слоев: 3**

**Кол-во нейронов: 13/39/65/91**

**Кол-во эпох: 5/50/100/1000**

**Оптимизатор: adam**

In [ ]:
for n_epochs in (5, 50, 100, 1000):
  for n_neuron in (13, 39, 65, 91):
    x_input = keras.layers.Input(shape=(13,))
    x = keras.layers.Flatten()(x_input)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x_output = keras.layers.Dense(1, activation='relu')(x)
    model=keras.models.Model(x_input,x_output)

    model.compile(
      optimizer='adam',
      loss='mse',
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history=model.fit(X_train, y_train, epochs=n_epochs)
    no_layers= 3

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    MAE = history.history['mean_absolute_error'][-1]
    MSE = history.history['loss'][-1]

    df = df.append({'no. layers': no_layers, 'no. neurons': n_neuron, 'no. epochs': n_epochs,'optimizer': 'adam', 'MAE': MAE, 'MSE': MSE}, ignore_index=True)

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 501/1000
12/12 [==============================] - 0s 3ms/step - loss: 19.6160 - mean_absolute_error: 3.0498
Epoch 502/1000
12/12 [==============================] - 0s 2ms/step - loss: 19.6642 - mean_absolute_error: 3.0223
Epoch 503/1000
12/12 [==============================] - 0s 2ms/step - loss: 19.6244 - mean_absolute_error: 3.0307
Epoch 504/1000
12/12 [==============================] - 0s 3ms/step - loss: 19.6353 - mean_absolute_error: 3.0456
Epoch 505/1000
12/12 [==============================] - 0s 3ms/step - loss: 19.8413 - mean_absolute_error: 3.0242
Epoch 506/1000
12/12 [==============================] - 0s 3ms/step - loss: 19.6318 - mean_absolute_error: 3.0494
Epoch 507/1000
12/12 [==============================] - 0s 3ms/step - loss: 19.6361 - mean_absolute_error: 3.0707
Epoch 508/1000
12/12 [==============================] - 0s 3ms/step - loss: 19.6179 - mean_absolute_error: 3.0244
Epoch 509/1000
12/12

In [ ]:
df

,no. layers,no. neurons,no. epochs,optimizer,MAE,MSE
0,3,13,5,adam,21.643343,559.645935
1,3,39,5,adam,20.638145,513.679932
2,3,65,5,adam,20.736820,522.560120
3,3,91,5,adam,19.646387,479.708557
4,3,13,50,adam,8.321194,116.603065
5,3,39,50,adam,22.608707,596.461853
6,3,65,50,adam,5.245023,55.088844
7,3,91,50,adam,22.608707,596.461914
8,3,13,100,adam,5.374418,58.706646
9,3,39,100,adam,4.436654,38.996777


**Кол-во слоев: 3**

**Кол-во нейронов: 13/39/65/91**

**Кол-во эпох: 5/50/100/1000**

**Оптимизатор: SGD**

In [ ]:
for n_epochs in (5, 50, 100, 1000):
  for n_neuron in (13, 39, 65, 91):
    x_input = keras.layers.Input(shape=(13,))
    x = keras.layers.Flatten()(x_input)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x_output = keras.layers.Dense(1, activation='relu')(x)
    model=keras.models.Model(x_input,x_output)

    model.compile(
      optimizer='SGD',
      loss='mse',
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history=model.fit(X_train, y_train, epochs=n_epochs)
    no_layers= 3

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    MAE = history.history['mean_absolute_error'][-1]
    MSE = history.history['loss'][-1]

    df = df.append({'no. layers': no_layers, 'no. neurons': n_neuron, 'no. epochs': n_epochs,'optimizer': 'SGD', 'MAE': MAE, 'MSE': MSE}, ignore_index=True)

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 501/1000
12/12 [==============================] - 0s 3ms/step - loss: 5.4594 - mean_absolute_error: 1.8071
Epoch 502/1000
12/12 [==============================] - 0s 3ms/step - loss: 6.1035 - mean_absolute_error: 1.8571
Epoch 503/1000
12/12 [==============================] - 0s 4ms/step - loss: 4.8335 - mean_absolute_error: 1.6791
Epoch 504/1000
12/12 [==============================] - 0s 3ms/step - loss: 4.9950 - mean_absolute_error: 1.7132
Epoch 505/1000
12/12 [==============================] - 0s 3ms/step - loss: 5.3140 - mean_absolute_error: 1.7761
Epoch 506/1000
12/12 [==============================] - 0s 2ms/step - loss: 4.3499 - mean_absolute_error: 1.5866
Epoch 507/1000
12/12 [==============================] - 0s 2ms/step - loss: 4.6311 - mean_absolute_error: 1.6025
Epoch 508/1000
12/12 [==============================] - 0s 6ms/step - loss: 5.1541 - mean_absolute_error: 1.7708
Epoch 509/1000
12/12 [======

In [ ]:
df

,no. layers,no. neurons,no. epochs,optimizer,MAE,MSE
0,3,13,5,adam,21.643343,559.645935
1,3,39,5,adam,20.638145,513.679932
2,3,65,5,adam,20.736820,522.560120
3,3,91,5,adam,19.646387,479.708557
4,3,13,50,adam,8.321194,116.603065
5,3,39,50,adam,22.608707,596.461853
6,3,65,50,adam,5.245023,55.088844
7,3,91,50,adam,22.608707,596.461914
8,3,13,100,adam,5.374418,58.706646
9,3,39,100,adam,4.436654,38.996777


**Кол-во слоев: 8**

**Кол-во нейронов: 13/39/65/91**

**Кол-во эпох: 5/50/100/1000**

**Оптимизатор: adam**

In [ ]:
for n_epochs in (5, 50, 100, 1000):
  for n_neuron in (13, 39, 65, 91):
    x_input = keras.layers.Input(shape=(13,))
    x = keras.layers.Flatten()(x_input)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x_output = keras.layers.Dense(1, activation='relu')(x)
    model=keras.models.Model(x_input,x_output)

    model.compile(
      optimizer='adam',
      loss='mse',
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history=model.fit(X_train, y_train, epochs=n_epochs)
    no_layers= 8

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    MAE = history.history['mean_absolute_error'][-1]
    MSE = history.history['loss'][-1]

    df = df.append({'no. layers': no_layers, 'no. neurons': n_neuron, 'no. epochs': n_epochs,'optimizer': 'adam', 'MAE': MAE, 'MSE': MSE}, ignore_index=True)

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 501/1000
12/12 [==============================] - 0s 8ms/step - loss: 1.3433 - mean_absolute_error: 0.8681
Epoch 502/1000
12/12 [==============================] - 0s 6ms/step - loss: 1.4876 - mean_absolute_error: 0.9339
Epoch 503/1000
12/12 [==============================] - 0s 7ms/step - loss: 1.8146 - mean_absolute_error: 1.0324
Epoch 504/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.6107 - mean_absolute_error: 0.9873
Epoch 505/1000
12/12 [==============================] - 0s 6ms/step - loss: 1.5852 - mean_absolute_error: 0.9561
Epoch 506/1000
12/12 [==============================] - 0s 7ms/step - loss: 1.4289 - mean_absolute_error: 0.9069
Epoch 507/1000
12/12 [==============================] - 0s 6ms/step - loss: 1.5265 - mean_absolute_error: 0.9294
Epoch 508/1000
12/12 [==============================] - 0s 12ms/step - loss: 1.4411 - mean_absolute_error: 0.8914
Epoch 509/1000
12/12 [=====

In [ ]:
df

,no. layers,no. neurons,no. epochs,optimizer,MAE,MSE
0,3,13,5,adam,21.643343,559.645935
1,3,39,5,adam,20.638145,513.679932
2,3,65,5,adam,20.736820,522.560120
3,3,91,5,adam,19.646387,479.708557
4,3,13,50,adam,8.321194,116.603065
5,3,39,50,adam,22.608707,596.461853
6,3,65,50,adam,5.245023,55.088844
7,3,91,50,adam,22.608707,596.461914
8,3,13,100,adam,5.374418,58.706646
9,3,39,100,adam,4.436654,38.996777


**Кол-во слоев: 8**

**Кол-во нейронов: 13/39/65/91**

**Кол-во эпох: 5/50/100/1000**

**Оптимизатор: SGD**

In [ ]:
for n_epochs in (5, 50, 100, 1000):
  for n_neuron in (13, 39, 65, 91):
    x_input = keras.layers.Input(shape=(13,))
    x = keras.layers.Flatten()(x_input)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x = keras.layers.Dense(n_neuron, activation='relu')(x)
    x_output = keras.layers.Dense(1, activation='relu')(x)
    model=keras.models.Model(x_input,x_output)

    model.compile(
      optimizer='SGD',
      loss='mse',
      metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history=model.fit(X_train, y_train, epochs=n_epochs)
    no_layers= 8

    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    MAE = history.history['mean_absolute_error'][-1]
    MSE = history.history['loss'][-1]

    df = df.append({'no. layers': no_layers, 'no. neurons': n_neuron, 'no. epochs': n_epochs,'optimizer': 'SGD', 'MAE': MAE, 'MSE': MSE}, ignore_index=True)

Выходные данные были обрезаны до нескольких последних строк (5000).
Epoch 501/1000
12/12 [==============================] - 0s 3ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoch 502/1000
12/12 [==============================] - 0s 3ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoch 503/1000
12/12 [==============================] - 0s 3ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoch 504/1000
12/12 [==============================] - 0s 5ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoch 505/1000
12/12 [==============================] - 0s 4ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoch 506/1000
12/12 [==============================] - 0s 4ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoch 507/1000
12/12 [==============================] - 0s 4ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoch 508/1000
12/12 [==============================] - 0s 4ms/step - loss: 596.4619 - mean_absolute_error: 22.6087
Epoc

In [ ]:
df

,no. layers,no. neurons,no. epochs,optimizer,MAE,MSE
0,3,13,5,adam,21.643343,559.645935
1,3,39,5,adam,20.638145,513.679932
2,3,65,5,adam,20.736820,522.560120
3,3,91,5,adam,19.646387,479.708557
4,3,13,50,adam,8.321194,116.603065
...,...,...,...,...,...,...
59,8,91,100,SGD,22.608707,596.461975
60,8,13,1000,SGD,6.848386,86.846375
61,8,39,1000,SGD,22.608706,596.461853
62,8,65,1000,SGD,22.608707,596.461914


**Выводы**

In [ ]:
df.sort_values(by='MAE', ascending=False)

,no. layers,no. neurons,no. epochs,optimizer,MAE,MSE
32,8,13,5,adam,22.608707,596.461853
21,3,39,50,SGD,22.608707,596.461853
27,3,91,100,SGD,22.608707,596.461853
38,8,65,50,adam,22.608707,596.461853
40,8,13,100,adam,22.608707,596.461853
...,...,...,...,...,...,...
44,8,13,1000,adam,1.534505,4.186487
30,3,65,1000,SGD,1.356610,3.222727
45,8,39,1000,adam,0.586358,0.614161
46,8,65,1000,adam,0.509810,0.468806


In [ ]:
adam_df = df[df['optimizer'].isin(['adam'])]
adam_df.sort_values(by='MAE', ascending=False)

,no. layers,no. neurons,no. epochs,optimizer,MAE,MSE
32,8,13,5,adam,22.608707,596.461853
12,3,13,1000,adam,22.608707,596.461853
43,8,91,100,adam,22.608707,596.461914
42,8,65,100,adam,22.608707,596.461914
40,8,13,100,adam,22.608707,596.461853
5,3,39,50,adam,22.608707,596.461853
38,8,65,50,adam,22.608707,596.461853
7,3,91,50,adam,22.608707,596.461914
0,3,13,5,adam,21.643343,559.645935
2,3,65,5,adam,20.736820,522.560120


In [ ]:
sgd_df = df[df['optimizer'].isin(['SGD'])]
sgd_df.sort_values(by='MAE', ascending=False)

,no. layers,no. neurons,no. epochs,optimizer,MAE,MSE
48,8,13,5,SGD,22.608707,596.461853
27,3,91,100,SGD,22.608707,596.461853
62,8,65,1000,SGD,22.608707,596.461914
59,8,91,100,SGD,22.608707,596.461975
57,8,39,100,SGD,22.608707,596.461853
56,8,13,100,SGD,22.608707,596.461853
54,8,65,50,SGD,22.608707,596.461914
52,8,13,50,SGD,22.608707,596.461853
51,8,91,5,SGD,22.608707,596.461853
50,8,65,5,SGD,22.608707,596.461853


1. Минимальные значения MAE и MSE замечены при испольщовании оптимизатора adam.
2. При увеличении количества нейронов, количества эпох и слоев показатели MAE и MSE улучшаются.
3. Лучшая комбинация, выбранная по показателю MAE: 8 слоев/ 39 нейронов/ 100 эпох / adam. При этой комбинации Средняя абсолютная ошибка = 1, 6544. Что не сильно отличается от показателя, если использовать 1000 эпох, но при этом занимает меньше времени.

К сожалению, не хватает навыков для построения красивых графиков, на основании полученных результатов.
И есть сомнения на счет малой величины MAE, не признак ли это переобучения?

